In [14]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [15]:
# import ikn.csv
ikn = pd.read_csv('ikn.csv', sep=';')
ikn.head()

,Tanggal,Username,Clean Dataset
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama ibu kota...
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu umum jokowi struktur...


In [16]:

# Read stopwords from file
with open('stopwords.txt', 'r') as stop_file:
    stopwords = stop_file.read().split('\n')

# Check for NaN values and replace them with an empty string
ikn['Clean Dataset'] = ikn['Clean Dataset'].replace(np.nan, '', regex=True)

# Remove non-letter characters
ikn['Clean Dataset'] = ikn['Clean Dataset'].str.replace('[^a-zA-Z]', ' ')

# Remove stopwords
ikn['Clean Dataset'] = ikn['Clean Dataset'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords]))

# Print the result
ikn.head()


,Tanggal,Username,Clean Dataset
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...


In [22]:
import nltk

# Download the vader_lexicon resource
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\GLOBAL
[nltk_data]     KOMPUTER\AppData\Roaming\nltk_data...


True

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import reprlib


# Assuming 'ikn' is your DataFrame with a 'Clean Dataset' column

# Your previous code for cleaning the 'Clean Dataset' column

# Initialize SentimentIntensityAnalyzers
sia1A, sia1B, sia2 = SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer(), SentimentIntensityAnalyzer()
sia1A.lexicon.clear()
sia1B.lexicon.clear()
sia2.lexicon.clear()

# Load custom lexicons
with open('leksikon/inset/_json_inset-neg.txt') as f:
    data1A = f.read()
with open('leksikon/inset/_json_inset-pos.txt') as f:
    data1B = f.read()
with open('leksikon/sentistrength_id/_json_sentiwords_id.txt') as f:
    data2 = f.read()

insetNeg = json.loads(data1A)
insetPos = json.loads(data1B)
senti = json.loads(data2)

# Update lexicons
sia1A.lexicon.update(insetNeg)
sia1B.lexicon.update(insetPos)
sia2.lexicon.update(senti)

# Apply sentiment analysis to the 'Clean Dataset' column
ikn['Sentiment1A'] = ikn['Clean Dataset'].apply(lambda x: sia1A.polarity_scores(x)['compound'])
ikn['Sentiment1B'] = ikn['Clean Dataset'].apply(lambda x: sia1B.polarity_scores(x)['compound'])
ikn['Sentiment2'] = ikn['Clean Dataset'].apply(lambda x: sia2.polarity_scores(x)['compound'])

# Map compound scores to labels (positive, neutral, negative)
ikn['Label1A'] = ikn['Sentiment1A'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))
ikn['Label1B'] = ikn['Sentiment1B'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))
ikn['Label2'] = ikn['Sentiment2'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))


In [25]:
# Print the result
ikn.head()

,Tanggal,Username,Clean Dataset,Sentiment1A,Sentiment1B,Sentiment2,Label1A,Label1B,Label2
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...,-0.9849,0.9776,0.7184,negative,positive,positive
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...,-0.9186,0.9325,0.0000,negative,positive,neutral
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...,-0.8750,0.9750,0.7184,negative,positive,positive
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,negative,positive,positive
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...,-0.9517,0.7906,0.0000,negative,positive,neutral


In [33]:
sample = "jokowi kok ada ada saja masa negara banyak hutang pengen bangun ibukota sih"
print("insetNeg: ", sia1A.polarity_scores(sample))
print("insetPos: ", sia1B.polarity_scores(sample))
print("insetCpdSum: 'compound':", sia1A.polarity_scores(sample)["compound"] + sia1B.polarity_scores(sample)["compound"])

print("senti\t: ", sia2.polarity_scores(sample))

insetNeg:  {'neg': 0.781, 'neu': 0.219, 'pos': 0.0, 'compound': -0.9799}
insetPos:  {'neg': 0.0, 'neu': 0.333, 'pos': 0.667, 'compound': 0.9638}
insetCpdSum: 'compound': -0.016100000000000003
senti	:  {'neg': 0.2, 'neu': 0.8, 'pos': 0.0, 'compound': -0.4588}


In [34]:
def is_positive_inset(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia1A.polarity_scores(tweet)["compound"] + sia1B.polarity_scores(tweet)["compound"] > 0

def is_positive_senti(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia2.polarity_scores(tweet)["compound"] > 0

In [39]:

#  Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari sentiwords_id
ikn['Label2'] = ikn['Clean Dataset'].apply(lambda x: 'positive' if is_positive_senti(x) else ('neutral' if sia2.polarity_scores(x)['compound'] == 0 else 'negative'))
# # Menulis hasil klasifikasi label untuk setiap kalimat berdasarkan nilai compound dari insetNeg dan insetPos

ikn['Label1A'] = ikn['Clean Dataset'].apply(lambda x: 'positive' if is_positive_inset(x) else ('neutral' if sia1A.polarity_scores(x)['compound'] == 0 else 'negative'))

ikn.head()

,Tanggal,Username,Clean Dataset,Sentiment1A,Sentiment1B,Sentiment2,Label1A,Label1B,Label2,Positive1A,Positive1B,Positive2
0,2022-02-27 12:00:01+00:00,HIMASIERA,himbrenews halo sobat siera minggu enak baca b...,-0.9849,0.9776,0.7184,negative,positive,positive,False,False,True
1,2022-02-27 09:23:52+00:00,oncombandung,bangun bedeng kali jagain lahan proyek btw bet...,-0.9186,0.9325,0.0000,positive,positive,neutral,True,True,False
2,2022-02-27 08:22:53+00:00,fadjroeL,temu dubes nur sultan kazakhstan nama kota nus...,-0.8750,0.9750,0.7184,positive,positive,positive,True,True,True
3,2022-02-27 04:04:47+00:00,Maandala_06,ikn indonesia resmi pindah dki jakarta nusanta...,-0.9719,0.9682,0.8750,negative,positive,positive,False,False,True
4,2022-02-27 03:09:00+00:00,tribunkaltim,kepala otorita ikn tunggu jokowi struktur bada...,-0.9517,0.7906,0.0000,negative,positive,neutral,False,False,False


In [41]:
# Menggunakan KNN dengan kernel linear sebagai classifier
# random_state merupakan pseudo randomization, supaya hasil yang didapat akan tetap sama setiap kali eksperimen diulang
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

X = ikn[['Sentiment1A', 'Sentiment1B', 'Sentiment2']]
y = ikn['Label1A']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)    



KNeighborsClassifier()

In [42]:

# Memprediksi label dari data test
y_pred = classifier.predict(X_test)

In [44]:

# Membuat confusion matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[484   0  15]
 [  0  51   0]
 [ 14   0 300]]
              precision    recall  f1-score   support

    negative       0.97      0.97      0.97       499
     neutral       1.00      1.00      1.00        51
    positive       0.95      0.96      0.95       314

    accuracy                           0.97       864
   macro avg       0.97      0.98      0.97       864
weighted avg       0.97      0.97      0.97       864



In [45]:
# validasi k fold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score


# Membuat model KNN dengan k=5
classifier = KNeighborsClassifier(n_neighbors=5)

# Membuat objek KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Membuat list untuk menampung hasil akurasi validasi
acc_scores = []
prec_scores = []
rec_scores = []
f1_scores = []

# Melakukan validasi silang
for train_index, test_index in kf.split(X):
    # Membagi data train dan data test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Melatih model dengan data train
    classifier.fit(X_train, y_train)

    # Memprediksi data test
    y_pred = classifier.predict(X_test)

    # Menghitung akurasi
    acc_score = classifier.score(X_test, y_test)
    acc_scores.append(acc_score)

    # Menghitung precision
    prec_score = precision_score(y_test, y_pred, average='macro')
    prec_scores.append(prec_score)

    # Menghitung recall
    rec_score = recall_score(y_test, y_pred, average='macro')
    rec_scores.append(rec_score)

    # Menghitung f1
    f1_score = f1_score(y_test, y_pred, average='macro')
    f1_scores.append(f1_score)

TypeError: 'numpy.float64' object is not callable